## Bločne šifre

In [ ]:
import os
import secrets
import cryptography
# pip install cryptography
#- varni (enostavni) del, ki od razvijalca ne terja veliko konfiguracijskih nastavitev in je primeren tudi za kriptografsko manj vešče uporabnike, in
#- nevarni (hazmat) del, kjer se pričakuje, da veste, kaj počnete.
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes    # AES-CTR
from cryptography.hazmat.primitives import padding  # PKCS7 padding za AES v načinu CBC

### Varni PRG

In [ ]:
key = secrets.token_bytes(16)
iv = secrets.token_bytes(16)

key = os.urandom(16)
iv = os.urandom(16)

## Šifriranje AES-CTR

In [ ]:
"""
Za šifriranje potrebujemo
- sporočilo v čistopisu
- ključ
- IV
- šifro -- algoritem $E$
"""

def enc_aes_ctr(k: bytes, pt: bytes, iv: bytes=b"") -> tuple[bytes, bytes]:
    if not iv:
        iv = os.urandom(16)
    cipher = Cipher(algorithms.AES(k), modes.CTR(iv)).encryptor()
    
    ct = cipher.update(pt) + cipher.finalize()
    print(iv, ct)
    return iv, ct

"""
Dešifriranje je zelo podobno, potrebujemo:
- Tajnopis
- Ključ
- IV, ki je bil uporabljen pri šifriranju
- Šifro -- algoritem $D$
"""

def dec_aes_ctr(k: bytes, ct: bytes, iv: bytes) -> bytes:
    decript = Cipher(algorithms.AES(k), modes.CTR(iv)).decryptor()
    pt = decript.update(ct) + decript.finalize()
    return pt


##### Dodatek


Šifriranje je sedaj _enostavno_:

1. Vmesnik je zastavljen, da lahko šifriramo večkrat, npr. da nalagamo podatke iz več datotek in nimamo vsega v pomnilniku.
2. Ko pokličemo `encryptor.update(pt)`, se podan čistopis šifrira in nastali tajnopis vrne, a ne nujno ves: lahko se zgodi, da zadnji blok čistopisa ne sovpada z velikostjo bloka bločne šifre. V tem primeru se zadnji blok čistopisa še nahaja nešifriran v objektu `encryptor` in čaka morebitne nove podatke, da se blok zapolni.
3. Šifriranje zaključimo s klicem funkcije `encryptor.finalize()` -- takrat se ves morebitni čistopis v objektu `encryptor` šifrira in vrne.

Vse rezultate klicev funkcije `update(pt)` ter na koncu tudi klica `finalize()` dodajamo na seznam bajtov, ki predstavlja končni tajnopis. K temu moramo še dodati IV, sicer ne bo mogoče dešifrirati.

#### Šifriranje vsebine

In [ ]:
def read_and_encrypt_file(key: bytes, filename: str):
    iv = os.urandom(16)
    
    with open(filename, 'br') as h:     # br = binary read
            pt = h.read()
            
    cifer = Cipher(algorithms.AES(key), modes.CTR(iv)).encryptor()
    ct = cifer.update(pt) + cifer.finalize()
    
    with open(filename + ".enc", 'wb') as h:        # wb = write binary
        h.write(iv)
        h.write(ct)
        
        
def read_and_decrypt(key: bytes, filename: str) -> bytes:
    with open(filename, 'br') as h:
            data = h.read()
        
    iv, ct = data[:16], data[16:]
    cifer_d = Cipher(algorithms.AES(key), modes.CTR(iv)).decryptor()
    
    pt = cifer_d.update(ct) + cifer_d.finalize()
    return pt

### Gnetljivost tajnopisa

In [ ]:
email = "nandi@obvlada.si"

# Funkcija, ki spremeni e-poštni naslov v šifrov AES-CTR načinu
def change_ct(ct, new_email):
    b_new_email = new_email.encode("utf8")
    
    c = bytearray(ct)
    for i in range(len(b_new_email)):
        c[i] = ct[i] ^ (ord(email[i]) ^ b_new_email[i])
    return bytes(c)

## Šifriranje AES-CBC

In [ ]:
def enc_aes_cbc(k: bytes, pt: bytes, iv: bytes) -> tuple[bytes, bytes]:
    if not iv:
        iv = os.urandom(16)

    padder = padding.PKCS7(128).padder()
    padded = padder.update(pt) + padder.finalize()

    cipher = Cipher(algorithms.AES(k), modes.CBC(iv)).encryptor()
    ct = cipher.update(padded) + cipher.finalize()

    return iv, ct


def dec_aes_cbc(k: bytes, ct: bytes, iv: bytes) -> bytes:
    cipher = Cipher(algorithms.AES(k), modes.CBC(iv)).decryptor()
    ct_padded = cipher.update(ct) + cipher.finalize()
    
    unpadder = padding.PKCS7(128).unpadder()
    pt = unpadder.update(ct_padded) + unpadder.finalize()
    return pt

### Padding

In [ ]:
padder = padding.PKCS7(128).padder() # podlagamo do velikost 128 bitov
padded_data = padder.update(b"Tole bosta dva bloka podatkov.") # naš vhodni podatek
padded_data += padder.finalize() # zaključimo podlaganje

unpadder = padding.PKCS7(128).unpadder() # podloženo je za velikst bloka 128 bitov
unpadded_data = unpadder.update(padded_data) # naložimo podložene podatke
unpadded_data += unpadder.finalize() # zaključimo podlaganje